In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

In [4]:
import tensorflow as tf

# Force TensorFlow to use GPU 0
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Allow memory growth
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.set_visible_devices(gpus[0], 'GPU')
        print("GPU is set and memory growth is enabled.")
    except RuntimeError as e:
        print(e)

GPU is set and memory growth is enabled.


In [5]:
# Optional: check GPU
print("Available GPU:", tf.config.list_physical_devices('GPU'))

Available GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [7]:
# Set dataset paths
train_dir = '/kaggle/input/fer2013/train'
val_dir = '/kaggle/input/fer2013/test'

In [8]:
# Image generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    batch_size=64,
    color_mode='grayscale',
    class_mode='categorical'
)

val = val_datagen.flow_from_directory(
    val_dir,
    target_size=(48, 48),
    batch_size=64,
    color_mode='grayscale',
    class_mode='categorical'
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [9]:
# Define CNN model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 7 emotion classes
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1747404541.324072     205 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Save best model
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

# Train model
model.fit(train, validation_data=val, epochs=30, callbacks=[checkpoint])

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30


I0000 00:00:1747404547.521130     267 service.cc:148] XLA service 0x7cfc50002290 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747404547.521183     267 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1747404547.783105     267 cuda_dnn.cc:529] Loaded cuDNN version 90300


  4/449 ━━━━━━━━━━━━━━━━━━━━ 24s 54ms/step - accuracy: 0.1507 - loss: 1.9214

I0000 00:00:1747404549.523689     267 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.2409 - loss: 1.8319
Epoch 1: val_accuracy improved from -inf to 0.35539, saving model to best_model.h5
449/449 ━━━━━━━━━━━━━━━━━━━━ 51s 104ms/step - accuracy: 0.2409 - loss: 1.8318 - val_accuracy: 0.3554 - val_loss: 1.6568
Epoch 2/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.3333 - loss: 1.6732
Epoch 2: val_accuracy improved from 0.35539 to 0.43522, saving model to best_model.h5
449/449 ━━━━━━━━━━━━━━━━━━━━ 44s 98ms/step - accuracy: 0.3333 - loss: 1.6731 - val_accuracy: 0.4352 - val_loss: 1.4713
Epoch 3/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.4016 - loss: 1.5469
Epoch 3: val_accuracy improved from 0.43522 to 0.47228, saving model to best_model.h5
449/449 ━━━━━━━━━━━━━━━━━━━━ 44s 97ms/step - accuracy: 0.4016 - loss: 1.5469 - val_accuracy: 0.4723 - val_loss: 1.3854
Epoch 4/30
449/449 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.4357 - loss: 1.4782
Epoch 4: val_accuracy improved from 0.47228 to 0.4

In [11]:
model.save('emotion_model.h5')
